# Manual DB interventions
This notebook is used to manually intervene in the database, e.g. to fix inconsistencies.

Make a backup of the database before running this notebook, test on local/sandbox first, and use it with care!

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
from datetime import datetime, timedelta

import pandas as pd
import pytz

from shared.db.models import Metrics, RawFile

## Connect to the database

In [ ]:
# connect to the database
os.environ["MONGO_HOST"] = "localhost"
os.environ["MONGO_PORT"] = "27017"
os.environ["MONGO_USER"] = "mongo"
os.environ["MONGO_PASSWORD"] = ""

from shared.db import engine
from shared.db.engine import connect_db

print("database user", engine.DB_USER)
connect_db()

## Query data

In [ ]:
# example: get all raw files and metrics from the last 9 days
max_age_in_days = 9
min_created_at = pd.Timestamp(
    datetime.now(tz=pytz.UTC) - timedelta(days=max_age_in_days)
)

raw_files_db = RawFile.objects(created_at__gte=min_created_at)

metrics_db = Metrics.objects(raw_file__in=raw_files_db)

print(len(list(raw_files_db)), "raw files found")
print(len(list(metrics_db)), "metrics found")

In [ ]:
from typing import Any

from mongoengine import QuerySet


def augment_raw_files_with_metrics(
    raw_files: QuerySet,
) -> dict[str, Any]:
    """Augment raw files with their latest metrics.

    Args:
        raw_files (QuerySet): A mongoengine QuerySet of RawFile objects to augment with metrics.

    Returns:
        dict[str, Any]: A dictionary containing raw file information and their latest metrics.
                              Each dictionary has the keys:
                              - "raw_file": A dictionary with raw file details.
                              - "metrics_{type}": A dictionary with the latest metrics of type "type" or an empty dictionary if none exist.

    """
    raw_files_dict: dict = {
        raw_file_mongo["_id"]: raw_file_mongo
        for raw_file in raw_files
        if (
            raw_file_mongo := dict(raw_file.to_mongo())
        )  # if condition is always true, but avoids double-calling to_mongo((
    }

    # querying all metrics at once to avoid load on DB
    for metrics_ in Metrics.objects.filter(
        raw_file__in=list(raw_files_dict.keys())
    ).order_by("-created_at_"):
        metrics = dict(metrics_.to_mongo())
        raw_files_dict[metrics["raw_file"]][f"metrics_{metrics['type']}"] = metrics

    return raw_files_dict


raw_files_with_metrics = augment_raw_files_with_metrics(raw_files_db)

## Helper methods

In [ ]:
import re


def starts_with_collision_flag(text: str) -> bool:
    """Check if a string starts with a collision flag, i.e. a date in format YYYYMMDD-HHMMSS-<6 digits milliseconds>."""
    pattern = r"^\d{8}-\d{6}-\d{6}-"
    return bool(re.match(pattern, text))

## Example: change file_info keys

### Select data

In [ ]:
ids = []
raw_files = []

for ff in raw_files_db:
    f = ff.to_mongo()
    if starts_with_collision_flag(f["_id"]):
        print(f["_id"])
        ids.append(f["_id"])
        raw_files.append(f)

### Write to database

In [ ]:
write_to_db = False  # set to True to save changes

for id_ in ids:
    raw_file = (RawFile.objects(id=id_)).first()

    new_file_info = {}

    for k, v in raw_file.file_info.items():
        new_k = k.replace(raw_file.original_name, raw_file.id)
        new_file_info[new_k] = v

    raw_file.file_info = new_file_info
    if write_to_db:
        raw_file.save()